## Series Temporales con quandl & Facebook Prophet

La predicción de series temporales puede ser algo complejo. Esta tarea se facilita con el paquete Prophet de facebook. A continuación, vamos a explorar las acciones de la empresa Tesla (TSLA) del 2011 al 2018. Vamos a ver lo sencillo que es crear una proyección para el 2019.

Para descargar los datos de TESLA, lo podemos hacer a traves del API de quandl. Obtener un key es gratuito y podemos realizarlo en https://www.quandl.com/. Para obtener una llave sin costo, registrese como estudiante con su cuenta de ucenfotec. Debe activar el link en su correo para que funcione.

In [ ]:
quandl_key = 'mzbz-NvFsKvi5nu9uZ4x'
#!pip3 install quandl

In [ ]:
import quandl
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# obtener acciones de TESLA de quandl
quandl.ApiConfig.api_key = quandl_key
data = quandl.get('WIKI/TSLA')

data.head()

### Cargar los datos de TESLA 

In [ ]:
plt.plot(data.index, data['Adj. Close'], 'r')
plt.title('Tesla Stock al Cierre')
plt.ylabel('Precio ($)');
plt.show();

### Instalar Prophet y crear el dataframe

In [ ]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '16'
os.environ['NUMEXPR_NUM_THREADS'] = '8'
import numexpr as ne

In [ ]:
# https://facebook.github.io/prophet/docs/quick_start.html
# https://en.wikipedia.org/wiki/Additive_model
#!pip3 install fbprophet
import fbprophet as pf

In [ ]:
# vamos a crear un nuevo dataframe para simplificar el analisis
tesla = pd.DataFrame({'ds': data.index.values, 'y': data['Adj. Close']})
tesla.head()

In [ ]:
# si hay error upgrade pystan 2.19.1.1
# model fitting!
m = pf.Prophet(daily_seasonality=True)
m.fit(tesla)

In [ ]:
# vamos a crear la prediccion a futuro con intervalos de confianza.
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig = m.plot(forecast)

### Desplegar Prediccion y Componentes

In [ ]:
fig2 = m.plot_components(forecast)

### Visualizacion Interactiva con Plotly

In [ ]:
from fbprophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [ ]:
# componentes intectactivos con plotly
plot_components_plotly(m, forecast)

## Resampling & Rolling Windows

In [ ]:
resample_weekly = data['Adj. Close'].resample('Y').mean()

plt.plot(tesla.index, data['Adj. Close'], 'r')
plt.plot(resample_weekly)
plt.title('Tesla Stock al Cierre (Resampled)')
plt.ylabel('Precio ($)');
plt.show();

In [ ]:
window_365d = data['Adj. Close'].rolling(window=100, center=True).mean()

plt.plot(data.index, data['Adj. Close'], 'r')
plt.plot(window_365d)
plt.title('Tesla Stock al Cierre (Rolling Window)')
plt.ylabel('Precio ($)');
plt.show();

## Auto-Regression

El método de autorregresión (AR) modela el siguiente paso en la secuencia como una función lineal de las observaciones en los pasos de tiempo anteriores.

La notación del modelo implica especificar el orden del modelo p como parámetro de la función AR, p. Ej. AR (p). Por ejemplo, AR (1) es un modelo de autorregresión de primer orden.

El método es adecuado para series de tiempo univariadas sin tendencia ni componentes estacionales.

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/7252143f217525cfbf556ff59ebafacf13e638cb" />

In [ ]:
# https://www.statsmodels.org/stable/generated/statsmodels.tsa.ar_model.AutoReg.html

from statsmodels.tsa.ar_model import AutoReg
x = data['Adj. Close'].values

# modelo
model = AutoReg(x, lags=1, seasonal=True, period=365)
model_fit = model.fit()

# prediccion
predict = model_fit.predict(
    start=len(x), 
    end=len(x) + len(x)-1, 
    dynamic=True)

plt.plot(data.index, data['Adj. Close'], 'r')
plt.plot(data.index, predict)
plt.title('Tesla Stock al Cierre (Prediccion: AR)')
plt.ylabel('Precio ($)');
plt.show();